In [0]:
res_json_data = spark.read.format('json').option('header', 'true').option('inferSchema', 'true').option('mode', 'permissive')\
    .load('/FileStore/tables/resturant_json_data.json')

res_json_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
%fs
ls /FileStore/tables/resturant_json_data.json

path,name,size,modificationTime
dbfs:/FileStore/tables/resturant_json_data.json,resturant_json_data.json,669503,1745497396000


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

ex_data = res_json_data.select('*', explode('restaurants').alias('new_added_restaurants'))  # converted array in struct
ex_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
drop_data = ex_data.drop('restaurants')
drop_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- new_added_restaurants: struct (nullable = true)
 |    |-- restaurant: struct (nullable = true)
 |    |    |-- R: struct (nullable = true)
 |    |    |    |-- res_id: long (nullable = true)
 |    |    |-- apikey: string (nullable = true)
 |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |-- cuisines: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- deeplink: string (nullable = true)
 |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- events_url: string (nullable = true)
 |    |    |-- featured_image: string (nullable = true)
 |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |-- h

In [0]:
select_id_data = drop_data.select('new_added_restaurants.restaurant.R.res_id')
select_id_data.show()

+--------+
|  res_id|
+--------+
|17066603|
|17059541|
|17064405|
|17057797|
|17057591|
|17064266|
|17060516|
|17060320|
|17059060|
|17059012|
|17060869|
|17061231|
|17058534|
|17057925|
|17064031|
|17061237|
|17061253|
|17061296|
|17061205|
|17057397|
+--------+
only showing top 20 rows



In [0]:
est_data = drop_data.select('*', 'new_added_restaurants.restaurant.R.res_id', 'new_added_restaurants.restaurant.name', explode_outer('new_added_restaurants.restaurant.establishment_types').alias("new_added_e_types")).drop('new_added_restaurants.restaurant.establishment_types').drop('new_added_restaurants', 'message', 'results_found')
est_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- res_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- new_added_e_types: string (nullable = true)



In [0]:
est_data = drop_data.select('*', 'new_added_restaurants.restaurant.R.res_id', 'new_added_restaurants.restaurant.name', explode_outer('new_added_restaurants.restaurant.establishment_types').alias("new_added_e_types")).drop('new_added_restaurants.restaurant.establishment_types').drop('new_added_restaurants', 'message', 'results_found')
est_data.show()

+----+-------------+-------------+------+--------+--------------------+-----------------+
|code|results_shown|results_start|status|  res_id|                name|new_added_e_types|
+----+-------------+-------------+------+--------+--------------------+-----------------+
|null|           20|            1|  null|17066603|            The Coop|             null|
|null|           20|            1|  null|17059541|Maggiano's Little...|             null|
|null|           20|            1|  null|17064405|Tako Cheena by Po...|             null|
|null|           20|            1|  null|17057797|Bosphorous Turkis...|             null|
|null|           20|            1|  null|17057591|Bahama Breeze Isl...|             null|
|null|           20|            1|  null|17064266|Hawkers Asian Str...|             null|
|null|           20|            1|  null|17060516|Seasons 52 Fresh ...|             null|
|null|           20|            1|  null|17060320|Raglan Road Irish...|             null|
|null|    